In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, MaxPool2D, Input, Softmax, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
import os
import cv2
import keras

Using TensorFlow backend.


In [2]:
class BloodTypeModel():
    
    def __init__(self):
        self.model = Sequential()
        
        self.x_train = None
        self.y_train = None
        self.pre_x_train = None
        self.cat_y_train = None
        
        self.x_test = None
        self.y_test = None
        self.pre_x_test = None
        self.cat_y_test = None
        
        self.x_val = None
        self.y_val = None
        self.pre_x_val = None
        self.cat_y_val = None
        
        self.stdScl = StandardScaler()
        self.new_size = (80,60)
        
    def preprocess_img(self,img):
        image = cv2.resize(cv2.imread(img),self.new_size)
        image = np.asarray(image)
        return image
    
    def alpha_to_num(self,x,label):
        mapping = {}
        for i,v in enumerate(label):
            mapping[v]=i
            
        return mapping[x]
    
    def preprocess_data(self,url,smp):
        file_img = os.listdir(url)
        label = [x.lower() for x in file_img]
        img_links = []
        train = []
        category = []
        for x in file_img:
            img_list=os.listdir(url+'/'+x)
            c=0
            for img in img_list:
                print(url+'/'+x+'/'+img)
                train.append(self.preprocess_img(url+'/'+x+'/'+img))
                category.append(self.alpha_to_num(x.lower(),label))
#                 if(c>=smp):
#                     break
#                 c+=1
                
        return np.asarray(train),category
    
    def preprocess_train_set(self,url,smp=10):
        self.x_train,self.y_train = self.preprocess_data(url,smp)
        self.cat_y_train = to_categorical(self.y_train,num_classes = 4)
    
    def preprocess_test_set(self,url,smp=10):
        self.x_test,self.y_test = self.preprocess_data(url,smp)
        self.cat_y_test = to_categorical(self.y_test,num_classes = 4)
    
    def preprocess_val_set(self,url,smp=10):
        self.x_val,self.y_val = self.preprocess_data(url,smp)
        self.cat_y_val = to_categorical(self.y_val,num_classes = 4)
        
    def build_model(self):
        input_shape = (60, 80, 3)
        
        self.model.add(Conv2D(32, (11,11), padding="same",activation="relu",input_shape=input_shape))
        self.model.add(Conv2D(32, (7,7), padding="valid",activation="relu",input_shape=input_shape))
        self.model.add(MaxPool2D(pool_size=(2, 2)))
        
        self.model.add(Conv2D(32, (5, 5), padding="same",activation="relu"))
        self.model.add(Conv2D(32, (5, 5), padding="valid",activation="relu"))
        self.model.add(MaxPool2D(pool_size=(2, 2)))
        
        self.model.add(Conv2D(64, (3, 3), padding="same",activation="relu"))
        self.model.add(Conv2D(64, (3, 3), padding="valid",activation="relu"))
        self.model.add(MaxPool2D(pool_size=(2, 2)))
        self.model.add(BatchNormalization())
        self.model.add(Flatten())
        
        self.model.add(Dense(1024,activation="relu"))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(4,activation="softmax"))
        
        self.model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    def train_model(self,epochs=10):
#         print(self.x_train.shape)
        self.model.fit(
                        self.x_train,
                        self.cat_y_train,
                        epochs = epochs,
                        batch_size = 2,
                        validation_data = (self.x_test,self.cat_y_test)
                       )
        
    def save_model(self):
        self.model.save("weights.hdf5")
    
    def load_model(self):
        self.model.load("weights.hdf5")
    
    

In [ ]:
if __name__ == "__main__":
    btm = BloodTypeModel()
    btm.preprocess_train_set("data/images/TRAIN")
    btm.preprocess_test_set("data/images/TEST")
    btm.preprocess_val_set("data/images/TEST_SIMPLE")
    btm.build_model()
    btm.train_model(50)
    btm.load_model()